# Connect to AWS, Create Security Groups, Access Key Pairs and Spin Up & Manage an EC2 Instance

## Note: 
Note the prettiest, cleanest or really best code but hey... it works and you can't take that away from it.

What does it do? 
- Creates a security that allows for ssh and http 
- Creates a key pair if one does not already exist 
- Creates two EC2 instances within different availibility zones within a region 
- Creates a load balancer 
- Creates a target group that points to the ec2 instances 
- Then has some useful functions for getting info, stopping, resuming and terminating instances 




## Good blogs that help:
Obviously the Boto3 Doco 

https://www.stratoscale.com/knowledge/load-balancing/aws-elb/boto-3-for-elb/example-work-with-a-load-balancer-and-target-group/ 

http://boto.cloudhackers.com/en/latest/elb_tut.html


# Import Python Libraries

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError 

# Create Security Group & Authorize Traffic

Create a security group that allows ssh and http traffic 

In [ ]:
group_name = "21484971-sg"
security_group_description="SHH traffic only - 21484971"
availability_zone = "ap-southeast-2"

## Rsource & Client Connection
ec2 = boto3.resource('ec2', availability_zone)
ec2_client = boto3.client("ec2", region_name=availability_zone)

# Try get the security group by name 
try:
    response = ec2_client.describe_security_groups(GroupNames=[group_name])
    for p in response['SecurityGroups']:
        print("Group Name: " + p['GroupName'])
        print("Group Description: " + p['Description'])
        print("GroupId: " + p['GroupId'])
        security_group_id = p['GroupId']
except ClientError as e: #Fails when the group does not exist 
    # Try to create security group with the provided id, catch error any errors
    try:
        # Create a security group and allow SSH inbound rule through the VPC
        securitygroup = ec2.create_security_group(GroupName=group_name, Description=security_group_description)
        securitygroup.authorize_ingress(CidrIp='0.0.0.0/0', IpProtocol='tcp', FromPort=22, ToPort=22)
        securitygroup.authorize_ingress(CidrIp='0.0.0.0/0', IpProtocol='tcp', FromPort=80, ToPort=80)
        
        security_group_id = securitygroup.id
    
    except ClientError as e:
        print(e)

    print(security_group_id)


# Create Key Pair if not created if not, create one

In [ ]:
ec2_resources = boto3.resource('ec2')

key_file_name = "21484971-key.pem"
key_name = '21484971-key'

path = "./" + key_file_name

if( not os.path.isfile(path)):
    # create a file to store the key locally
    outfile = open(key_file_name,'w')

    # call the boto ec2 function to create a key pair
    key_pair = ec2_resources.create_key_pair(KeyName=key_name)

    # capture the key and store it in a file
    KeyPairOut = str(key_pair.key_material)
    print(KeyPairOut)

    outfile.write(KeyPairOut)

    ## Run Chmod 400 from python
    mode = os.stat(path).st_mode
    mode |= (mode & 400) >> 2    # copy R bits to X
    os.chmod(path, mode)


# 4) Create Connection, Instance & Tag in two different Availability
Not the cleanest or best way to do this but hey, it works so...

In [ ]:
## Instance information
image_id = "ami-d38a4ab1"
instance_type = "t2.micro"
security_group = security_group_id #manually override if not creating a group before (sg-091eb217391dc8774)
# key_name = '21484971-key' #Can manually override if not running in steps

availability_zone = "ap-southeast-2"
subnetid = "subnet-89792dd1"

## Rsource connection
ec2 = boto3.resource('ec2',availability_zone)

## Create instance
instances = ec2.create_instances(
    ImageId=image_id, 
    InstanceType = instance_type,  
    KeyName=key_name,
    MaxCount=1,
    MinCount=1,
   NetworkInterfaces = [
    {
        'SubnetId': subnetid,
        'DeviceIndex': 0,
        'AssociatePublicIpAddress': True,
        'Groups': [security_group]

    }]
 )

## Store and return instance id
instance_1_id = instances[0].instance_id
print(instance_1_id)


## Tag instance 
ec2.create_tags(Resources=[instance_1_id], Tags=[{'Key':'Name', 'Value':'21484971'}])

availability_zone_2 = "ap-southeast-2"
subnetid_2 = "subnet-2a20f762"

## Rsource connection
ec2 = boto3.resource('ec2',availability_zone_2)

## Create instance
instances = ec2.create_instances(
    ImageId=image_id, 
    InstanceType = instance_type,  
    KeyName=key_name,
    MaxCount=1,
    MinCount=1,
    NetworkInterfaces = [
    {
        'SubnetId': subnetid_2,
        'DeviceIndex': 0,
        'AssociatePublicIpAddress': True,
        'Groups': [security_group]

    }]
 )

## Store and return instance id
instance_2_id = instances[0].instance_id
print(instance_2_id)

## Tag instance 
ec2.create_tags(Resources=[instance_2_id], Tags=[{'Key':'Name', 'Value':'21484971'}])

# Create Load Balancer 

In [ ]:
 
 
#Load Balancer
elb = boto3.client('elbv2', availability_zone)

 # create load-balancer
create_lb_response = elb.create_load_balancer(Name='21484971',
                                                         Subnets=[subnetid, subnetid_2],
                                                         SecurityGroups=[security_group],
                                                         Scheme='internet-facing',
                                                          Tags=[
                                                                {
                                                                    'Key': 'Name',
                                                                    'Value': '21484971'
                                                                },
                                                            ],
                                                         )
print(create_lb_response)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        lbId = response['LoadBalancers'][0]['LoadBalancerArn']
        print ("Successfully created target group %s" % lbId)
else:
    print ("Create target group failed")

# Create Target Group

In [ ]:


response = elb.create_target_group(
    Name='21484971-tg',
    TargetType='instance',
    Protocol='HTTP',
    Port=80,
    VpcId= "vpc-38e71a5e", 
    Tags=[
        {
            'Key': 'Name',
            'Value': '21484971'
        },
    ]
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        tgId = response['TargetGroups'][0]['TargetGroupArn']
        print ("Successfully created target group %s" % tgId)
else:
    print ("Create target group failed")


# Register targets

targets_list = [dict(Id=target_id, Port=80) for target_id in [instance_1_id, instance_2_id]]

reg_targets_response = elb.register_targets(TargetGroupArn=tgId, Targets=targets_list)
# check register group returned successfully
if reg_targets_response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print ("Successfully registered targets")
else:
    print ("Register targets failed")

# Create Listener that binds the load balancer and target group 

In [ ]:
 # create Listener
create_listener_response = elb.create_listener(LoadBalancerArn=lbId,
                                                          Protocol='TCP', Port=80,
                                                          DefaultActions=[{'Type': 'forward',
                                                                           'TargetGroupArn': tgId}])

    # check create listener returned successfully
if create_listener_response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print ("Successfully created listener %s" % tgId)
else:
    print ("Create listener failed")

# 5) Get Ip Address

In [ ]:
ec2_client = boto3.client("ec2", region_name=availability_zone)
reservations = ec2_client.describe_instances(InstanceIds=[instance_1_id]).get("Reservations")
for reservation in reservations:
    for instance in reservation['Instances']:
        ip_address=instance.get("PublicIpAddress")

print(ip_address)

ec2_client = boto3.client("ec2", region_name=availability_zone_2)
reservations = ec2_client.describe_instances(InstanceIds=[instance_2_id]).get("Reservations")
for reservation in reservations:
    for instance in reservation['Instances']:
        ip_address=instance.get("PublicIpAddress")

print(ip_address)

# Use IP to Connect with SSH from terminal

In [ ]:
## Bash Command to run
# ssh -i <KEY FILE NAME>.pem ubuntu@<IP ADDRESS>

print(path)
print(instance_1_id)
print(instance_2_id)


# Instance Functions from Python

## Stop instance 

In [ ]:
response = ec2_client.stop_instances(InstanceIds=[instance_1_id])
print(response)
response = ec2_client.stop_instances(InstanceIds=[instance_2_id])
print(response)

## Resart Instance

In [ ]:
response = ec2_client.start_instances(InstanceIds=[instance_1_id])
print(response)
response = ec2_client.start_instances(InstanceIds=[instance_2_id])
print(response)

## Terminate Instance

In [ ]:
response = ec2_client.terminate_instances(InstanceIds=[instance_1_id])
print(response)
 
response = ec2_client.terminate_instances(InstanceIds=[instance_2_id])
print(response)